<a href="https://colab.research.google.com/github/diegolusa/cplusplus_poc/blob/main/ImpactaGov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Acesso ao GDrive

Acessar o diretório onde encontram-se os arquivos de despesas. No código a seguir está sendo utilizado o Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

#Instalar Spark

A biblioteca Spark permite fluxos de ETL e a aplicação de algoritmos de Machine Learning

In [ ]:
!pip install pyspark

# Processar arquivos pelo Spark

Leitura dos arquivos CSV de execução de despesas. Os arquivos foram baixados do portal de transparência (http://transparencia.gov.br/download-de-dados/despesas-execucao) e correspondem aos anos de 2019, 2020 e 2021.

In [ ]:
from pyspark.sql import SparkSession

dataset =  SparkSession\
  .builder\
  .appName("ImpactGov")\
  .getOrCreate()\
  .read\
  .format("csv")\
  .option("sep",";")\
  .option("inferSchema","true")\
  .option("header","true")\
  .option("ESCAPE quote", '"')\
  .option("encoding", "ISO-8859-1")\
  .load(f'/content/drive/MyDrive/ImpactaGov/dados_despesas')

print(f'Total de registros lidos: {dataset.count()}')


Adicionar coluna representando a natureza da despesa

In [ ]:
from pyspark.sql.functions import col,expr,concat, concat_ws
dataset = dataset.withColumn("Natureza",concat(col("Código Categoria Econômica"),col("Código Grupo de Despesa"),\
                                                    col("Código Modalidade da Despesa"),col("Código Elemento de Despesa")))
dataset = dataset.withColumn("Nome Natureza",concat_ws(" - ", col("Natureza"), col("Nome Elemento de Despesa")))

In [ ]:
print(dataset.filter(dataset["Natureza"] == "339039").count())

A ação 4572 refere-se a **CAPACITACAO DE SERVIDORES PUBLICOS FEDERAIS EM PROCESSO DE QUALIFICACAO E REQUALIFICACAO**

In [ ]:
#2018, 2019, 2020 e 2021
#tentar filtrar pela natureza de despesa 33903948/33904020

ds_acao_4572 = dataset.filter(dataset['Código Ação'] =='4572')
print(f'Total de registros lidos: {ds_acao_4572.count()}')

Após filtrar, os dados resultantes são exportados em formato CSV

In [ ]:
ds_acao_4572.coalesce(1).write.format("csv").mode("overwrite").option("header","true").save("/content/drive/MyDrive/ImpactaGov/output/resultado.csv")

Cálculo do valor líquido total investido a cada mês.

In [ ]:
from pyspark.sql.functions import col, regexp_replace, format_number, format_string
from pyspark.sql.types import DoubleType

total_por_ano_mes = ds_acao_4572\
  .withColumn("Valor Liquidado (R$)",regexp_replace("Valor Liquidado (R$)",",","."))\
  .withColumn("Valor Liquidado (R$)",col("Valor Liquidado (R$)").cast(DoubleType()))\
  .groupby('Ano e mês do lançamento')\
  .sum("Valor Liquidado (R$)")\
  .sort('Ano e mês do lançamento')

Plotar gráfico com os dados processados.

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
sns.set_theme(style="darkgrid")
sns.set(rc={"figure.figsize":(30, 8)})

In [ ]:
pandas_dataset = total_por_ano_mes.toPandas()
chart = sns.barplot(x='Ano e mês do lançamento',y='sum(Valor Liquidado (R$))', data=pandas_dataset)
chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')#rotacionar rótulos em X

plt.xlabel('Mês/Ano')
plt.ylabel('Investimento (Milhões)')
plt.title('Investimentos em Capacitação de Servidores (Ação 4572)')

plt.savefig("/content/drive/MyDrive/ImpactaGov/output/acao_4572.png")

Plotar gráfico por natureza de despesa (para cada ano)

In [ ]:
from pyspark.sql import functions as f
import pyspark

plt.rcParams["figure.figsize"] = [15, 15]
plt.rcParams["figure.dpi"] = 300
plt.rcParams["figure.autolayout"] = True


for ano in range(2018,2022):
  #agregar por natureza
  total_por_natureza  = ds_acao_4572\
    .withColumn("Valor Liquidado (R$)",regexp_replace("Valor Liquidado (R$)",",","."))\
    .withColumn("Valor Liquidado (R$)",col("Valor Liquidado (R$)").cast(DoubleType()))\
    .filter(col("Ano e mês do lançamento").startswith(str(ano)))\
    .groupby('Nome Natureza')\
    .agg(f.sum("Valor Liquidado (R$)").alias("total"))\
    .where("total > 0")\
    .sort(col('total').desc())
  #restringir a 7 naturezas principais  
  top_7_naturezas = total_por_natureza.limit(7)
  #obter as demais naturezas e consolidar em um único valor
  demais_naturezas = total_por_natureza.exceptAll(top_7_naturezas)
  total_demais_naturezas = demais_naturezas.agg(f.sum("total").alias("total")).collect()[0]["total"]
  consolidacao_demais_naturezas = SparkSession.builder.appName("consolidacaoDemaisNaturezas").getOrCreate()\
  .createDataFrame(data=[['Demais Naturezas',total_demais_naturezas]], schema=['Nome Natureza','total'])
  #unir as 7 principais com a consolidação das demais
  top_8_naturezas = top_7_naturezas.union(consolidacao_demais_naturezas)
  pandas_dataset  = top_8_naturezas.toPandas()
  plt.title(f'Natureza de Despesa ({ano})', loc='left')
  plt.pie(pandas_dataset["total"], labels=pandas_dataset['Nome Natureza'],\
        rotatelabels=True,\
        explode=[0,0,0,0,0,0,0,0.3],\
        colors= sns.color_palette("bright"),\
        autopct='%1.1f%%',\
        pctdistance=0.8,\
        startangle=90, shadow=True)
  plt.savefig(f'/content/drive/MyDrive/ImpactaGov/output/acao_4572_por_natureza_{ano}.png')
  plt.clf()
  plt.cla()
  plt.close()
